In [2]:
import json
import pandas as pd
import re
dict_mapping = {'Presenting Irrelevant Data (Red Herring)': 0,
 'Bandwagon': 1,
 'Smears': 2,
 'Glittering generalities (Virtue)': 3,
 'Causal Oversimplification': 4,
 'Whataboutism': 5,
 'Loaded Language': 6,
 'Exaggeration/Minimisation': 7,
 'Repetition': 8,
 'Thought-terminating cliché': 9,
 'Name calling/Labeling': 10,
 'Appeal to authority': 11,
 'Black-and-white Fallacy/Dictatorship': 12,
 'Obfuscation, Intentional vagueness, Confusion': 13,
 'Reductio ad hitlerum': 14,
 'Appeal to fear/prejudice': 15,
 "Misrepresentation of Someone's Position (Straw Man)": 16,
 'Flag-waving': 17,
 'Slogans': 18,
 'Doubt': 19,
 'Bandwagon_1': 20,
 'Whataboutism_1': 21,
 'Appeal to fear/prejudice_1': 22,
 'Flag-waving_1': 23,
 'Appeal to authority_1': 24,
 "Transfer_1" : 25,
 "Transfer" : 26,
 "Appeal to (Strong) Emotions" : 27
}
df_test_full = pd.read_csv('Merged_test_for_HypEmo_2a.csv').drop('Unnamed: 0',axis=1)
import numpy as np
loaded_array = np.load('softmax_9_test.npy')
mapping_dict = {1: 20, 5: 21, 15: 22, 17: 23, 11: 24,26 : 25}
for key in mapping_dict.keys():
    duplicate_row = mapping_dict[key]
    loaded_array[key] += loaded_array[int(duplicate_row)]

column_indices = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,26,27]
loaded_array = loaded_array[:,column_indices]
import matplotlib.pyplot as plt
abnormal_peaks_per_row = []
threshold = 1.5
for row in loaded_array:
    z_scores = np.abs((row - np.mean(row)) / np.std(row))
    abnormal_peaks = list(np.where(z_scores > threshold)[0])
    abnormal_peaks = [26 if x == 20 else x for x in abnormal_peaks]
    abnormal_peaks = [27 if x == 21 else x for x in abnormal_peaks]
    abnormal_peaks_per_row.append(abnormal_peaks)

final_list = []
idx2label = {idx: label for label, idx in dict_mapping.items()}
for x in abnormal_peaks_per_row:
    y = []
    for d in x:
        y.append(idx2label[d])
    final_list.append(y)

test_eng = df_test_full[:1500]
test_arab = df_test_full[1500:1500+120]
test_bulg = df_test_full[1500+120:1500+120+436]
test_mace = df_test_full[1500+120+436:]
#1000 + 1500 + 100 + 436 + 259

def json_saving(df,json_file_path):
    dict_list = []
    columns_to_include = ['labels', 'id']
    for index, row in df.iterrows():    
        row_dict = {column: row[column] for column in columns_to_include}
        dict_list.append(row_dict)
    json_string = json.dumps(dict_list,indent = 2)
    with open(json_file_path, 'w') as json_file:
        json_file.write(json_string)
    print(f'JSON data has been saved to: {json_file_path}')

json_saving(test_eng,"Hyp_emo_test_2a_eng.json")
json_saving(test_arab,"Hyp_emo_test_2a_arab.json")
json_saving(test_bulg,"Hyp_emo_test_2a_bulg.json")
json_saving(test_mace,"Hyp_emo_test_2a_mace.json")

    